This Notebook is recommended to be run in Kaggle Notebook with GPU Accelerator. The dataset is available in the link below. 

https://www.kaggle.com/andrewmvd/car-plate-detection

## 01_DataProcessingAndTraining

### Import libraries

In [47]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet
from glob import glob
from skimage import io, color
from shutil import copy

### Required Data

In [2]:
path = glob('/kaggle/input/car-plate-detection/annotations/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)
    
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/kaggle/input/car-plate-detection/annotations/...,209,283,135,169
1,/kaggle/input/car-plate-detection/annotations/...,191,242,147,169
2,/kaggle/input/car-plate-detection/annotations/...,115,277,115,153
3,/kaggle/input/car-plate-detection/annotations/...,36,62,175,186
4,/kaggle/input/car-plate-detection/annotations/...,71,215,205,246


In [4]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/kaggle/input/car-plate-detection/images',filename_image)
    return filepath_image

filename = df["filepath"][0]
print(getFilename(filename))

/kaggle/input/car-plate-detection/images/Cars339.png


In [5]:
# parsing
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'/kaggle/input/car-plate-detection/images/{name}'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height
df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height
0,/kaggle/input/car-plate-detection/annotations/...,209,283,135,169,/kaggle/input/car-plate-detection/images/Cars3...,500,300
1,/kaggle/input/car-plate-detection/annotations/...,191,242,147,169,/kaggle/input/car-plate-detection/images/Cars1...,400,268
2,/kaggle/input/car-plate-detection/annotations/...,115,277,115,153,/kaggle/input/car-plate-detection/images/Cars7...,400,267
3,/kaggle/input/car-plate-detection/annotations/...,36,62,175,186,/kaggle/input/car-plate-detection/images/Cars1...,400,221
4,/kaggle/input/car-plate-detection/annotations/...,71,215,205,246,/kaggle/input/car-plate-detection/images/Cars2...,517,303


Yolo need [class, center_x, center_y, w, h] refers to X and Y center position to the BB and with and height of the BB. 

In [6]:
# center_x, center_y, width , height
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,/kaggle/input/car-plate-detection/annotations/...,209,283,135,169,/kaggle/input/car-plate-detection/images/Cars3...,500,300,0.492000,0.506667,0.14800,0.113333
1,/kaggle/input/car-plate-detection/annotations/...,191,242,147,169,/kaggle/input/car-plate-detection/images/Cars1...,400,268,0.541250,0.589552,0.12750,0.082090
2,/kaggle/input/car-plate-detection/annotations/...,115,277,115,153,/kaggle/input/car-plate-detection/images/Cars7...,400,267,0.490000,0.501873,0.40500,0.142322
3,/kaggle/input/car-plate-detection/annotations/...,36,62,175,186,/kaggle/input/car-plate-detection/images/Cars1...,400,221,0.122500,0.816742,0.06500,0.049774
4,/kaggle/input/car-plate-detection/annotations/...,71,215,205,246,/kaggle/input/car-plate-detection/images/Cars2...,517,303,0.276596,0.744224,0.27853,0.135314


### Data Preparation

In [7]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16003, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 16003 (delta 21), reused 20 (delta 13), pack-reused 15967
Receiving objects: 100% (16003/16003), 14.66 MiB | 25.83 MiB/s, done.
Resolving deltas: 100% (10983/10983), done.


In [8]:
!pip install -r ./yolov5/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.5/616.5 kB 8.6 MB/s eta 0:00:0000:0100:01


In [9]:
mkdir /kaggle/working/yolov5/data_images/

In [10]:
mkdir /kaggle/working/yolov5/data_images/test/

In [11]:
mkdir /kaggle/working/yolov5/data_images/train/

In [12]:
### split the data into train and test
df_train = df.iloc[:200]
df_test = df.iloc[200:]

In [13]:
train_folder = './yolov5/data_images/train'

values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()

test_folder = './yolov5/data_images/test'

values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()

### training yolo

In [14]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()  

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=de1ffa1aab45d9c039eeb12f6e4b19202735682a424af08e2165bb7d613277d2
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  5% |  1% |
|  1 |  0% |  0% |


In [15]:
!python ./yolov5/train.py --data /kaggle/input/data-yaml-enr/data.yaml --cfg ./yolov5/models/yolov5s.yaml --batch-size 8 --name Model --epochs 100

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5/yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=/kaggle/input/data-yaml-enr/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1,

In [16]:
!python ./yolov5/export.py --weight ./yolov5/runs/train/Model/weights/best.pt --include torchscript onnx

export: data=yolov5/data/coco128.yaml, weights=['./yolov5/runs/train/Model/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
YOLOv5 🚀 v7.0-226-gdd9e338 Python-3.10.12 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from yolov5/runs/train/Model/weights/best.pt with output shape (1, 25200, 6) (13.8 MB)

TorchScript: starting export with torch 2.0.0...
TorchScript: export success ✅ 2.3s, saved as yolov5/runs/train/Model/weights/best.torchscript (27.2 MB)

ONNX: starting export with onnx 1.14.1...
================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
=====================